## To avoid bug, one needs  more  test 
In the following, we will examine the code of 2d Generalized Partition function to check its validation. Then we will use Cytnx to rewrite these functions. ( We will rewrite the function by naming start from `Re_` , but all the in/output will keep the same type as well as the same answer ) </br>


In [1]:
# import lib 

import numpy as np
from math import pi
from scipy import linalg
import time, itertools
import matplotlib.pyplot as plt
import HOTRG_two
import pandas as pd
import os
from pathlib import Path

### Functions in HOTRG_two.py
In the following we will test:
- Ising_exact : To calculate the rank-4 tensors (original pure and impurity tensors) which combine the partition function weight as well as (impurity) node tensors.
- merge_two : To merge two tensor with right index, If bond = 'x' , then the tensors will rotate 90 degree at first. And all the calculation keep the same 
- updat_pure : For the pure tensor, beside update the tensor we will also return the isometry tensor and the norm, which will be used for impurity tensor
- updat_impurity : For impurity tensor, will take the isometry and norm in `updat_pure`, by exerting the isometry tensors, update impurity tensor back to the original shape
- merge_four: calculate the result of merged 4 tensors, mainly used in the final tensor trace
- Cmax_CHmax: calculate the correlation function with largest distance $C_{max}$ and half largest max $C_{hmax}$ from the tensors

In [2]:
def Ising_exact(N,beta,dpp):

    Ten= np.zeros((N,N,N,N), dtype=dpp)
    iTen= np.zeros((N,N,N,N), dtype=dpp)

    for ii in range(N**4):
        Lab = dNb(N,ii,4)
        if np.mod(np.sum(Lab),N)==0:
            N0 = 0
            for jj in range(len(Lab)):
                if Lab[jj]==0: N0+=1

            NL = 4-N0
            VAL =np.exp(-2*beta)*( (np.sqrt(np.exp(beta*2.)+N-1))**(N0)*\
                (np.sqrt(np.exp(beta*2.)-1))**(NL) )/N

            Ten[Lab[0],Lab[1],Lab[2],Lab[3]]= VAL

        if np.mod(np.sum(Lab),N)==1:
            N0 = 0
            for jj in range(len(Lab)):
                if Lab[jj]==0: N0+=1

            NL = 4-N0
            VAL =np.exp(-2*beta)*( (np.sqrt(np.exp(beta*2.)+N-1))**(N0)*\
                (np.sqrt(np.exp(beta*2.)-1))**(NL) )/N

            iTen[Lab[0],Lab[1],Lab[2],Lab[3]]= VAL

    return Ten, iTen

def dNb(number,n,L):
    bb=[ ]
    for i in range(L-1,-1,-1):
        ii=number**i
        x,y= divmod(n, ii);n=y
        bb.append( x )
    return np.array(bb,dtype=int)

def Ising_square(T):
    N =2
    Tax = np.ones((N,N))
    taix = []
    DT = np.zeros((N,N,N,N))
    iDT = np.zeros((N,N,N,N))
    for ii in range(2):
        DT [ii,ii,ii,ii] = 1.0
        iDT [ii,ii,ii,ii] = -1.0
        iDT [0,0,0,0] = 1

    Tax [0,0] = np.exp((1./T))
    Tax [0,1] = np.exp((-1./T))
    Tax [1,0] = np.exp((-1./T))
    Tax [1,1] = np.exp((1./T))
    Tax[0,0] =Tax[0,0]*1.000
    Ya, Za = np.linalg.eigh(Tax)
    taix.append(np.tensordot(Za, np.diag(Ya**0.5),axes=(1,0)) )
    taix.append(np.tensordot(np.diag(Ya**0.5), (Za.conj().T) ,axes=(1,0)))


    DT =  np.tensordot( DT,taix[0],axes=(0,0))
    DT =  np.tensordot( DT,taix[0],axes=(0,0))
    DT =  np.tensordot( DT,taix[1],axes=(0,1))
    DT =  np.tensordot( DT,taix[1],axes=(0,1))

#        print '================='

    iDT =  np.tensordot(iDT,taix[0],axes=(0,0))
    iDT =  np.tensordot(iDT,taix[0],axes=(0,0))
    iDT =  np.tensordot(iDT,taix[1],axes=(0,1))
    iDT =  np.tensordot(iDT,taix[1],axes=(0,1))

    return DT, iDT
beta = 1
dpp = float
N = 2
Result1 = Ising_exact(N,beta,dpp)
Result2 = Ising_square(1)
print(Result1)
print(Result2)

(array([[[[4.76219569, 0.        ],
         [0.        , 3.62686041]],

        [[0.        , 3.62686041],
         [3.62686041, 0.        ]]],


       [[[0.        , 3.62686041],
         [3.62686041, 0.        ]],

        [[3.62686041, 0.        ],
         [0.        , 2.76219569]]]]), array([[[[0.        , 4.1559378 ],
         [4.1559378 , 0.        ]],

        [[4.1559378 , 0.        ],
         [0.        , 3.16513794]]],


       [[[4.1559378 , 0.        ],
         [0.        , 3.16513794]],

        [[0.        , 3.16513794],
         [3.16513794, 0.        ]]]]))
(array([[[[2.76219569, 0.        ],
         [0.        , 3.62686041]],

        [[0.        , 3.62686041],
         [3.62686041, 0.        ]]],


       [[[0.        , 3.62686041],
         [3.62686041, 0.        ]],

        [[3.62686041, 0.        ],
         [0.        , 4.76219569]]]]), array([[[[ 0.        , -3.16513794],
         [-3.16513794,  0.        ]],

        [[-3.16513794,  0.        ],
         

In [8]:
Lab = dNb(2,20,4)
print(Lab)

[2 1 0 0]


In [11]:
def dNb(number,n,L):
    bb=[ ]
    for i in range(L-1,-1,-1):
        ii=number**i
        x,y= divmod(n, ii);n=y
        bb.append( x )
    return np.array(bb,dtype=int)

def dNb2(number,n,L):
    bb=[ ]
    for i in range(L-1,-1,-1):
        ii=number**i
        x,y= divmod(n, ii)#;n=y
        bb.append( x )
    return np.array(bb,dtype=int)
tst1 = dNb(2,20,4)
tst2 = dNb2(2,20,4)
print(tst1)
print(tst2)

[2 1 0 0]
[ 2  5 10 20]


In [5]:
np.random.seed(1)
test = np.random.rand(10,10)
A,B  = linalg.eigh(test)
print(test)
print(A)
print(B)

[[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
  3.96767474e-01 5.38816734e-01]
 [4.19194514e-01 6.85219500e-01 2.04452250e-01 8.78117436e-01
  2.73875932e-02 6.70467510e-01 4.17304802e-01 5.58689828e-01
  1.40386939e-01 1.98101489e-01]
 [8.00744569e-01 9.68261576e-01 3.13424178e-01 6.92322616e-01
  8.76389152e-01 8.94606664e-01 8.50442114e-02 3.90547832e-02
  1.69830420e-01 8.78142503e-01]
 [9.83468338e-02 4.21107625e-01 9.57889530e-01 5.33165285e-01
  6.91877114e-01 3.15515631e-01 6.86500928e-01 8.34625672e-01
  1.82882773e-02 7.50144315e-01]
 [9.88861089e-01 7.48165654e-01 2.80443992e-01 7.89279328e-01
  1.03226007e-01 4.47893526e-01 9.08595503e-01 2.93614148e-01
  2.87775339e-01 1.30028572e-01]
 [1.93669579e-02 6.78835533e-01 2.11628116e-01 2.65546659e-01
  4.91573159e-01 5.33625451e-02 5.74117605e-01 1.46728575e-01
  5.89305537e-01 6.99758360e-01]
 [1.02334429e-01 4.14055988e-01 6.94400158e-01 4.14179270e

# with python version 3.8.3
`A` = [-1.59724462 -1.2482207  -0.84742532 -0.43437282  0.1460884   0.25099714
      0.67921396  0.9421721   1.33980598  5.16655232] <br>

`B` = [[ 0.46984916  0.27188827 -0.16878628  0.19586574 -0.30745463  0.35450665
   0.25701667 -0.04850955 -0.51049311 -0.29874582] <br>
 [ 0.13670125  0.06573782  0.614343   -0.21133182 -0.07846954  0.17808375
  -0.25276053 -0.53349393  0.18535656 -0.36630703]<br>
 [-0.48871955  0.14922369 -0.20456823  0.12578904 -0.57443908 -0.43159689
   0.00122234 -0.20707654  0.01635416 -0.34901098]<br>
 [ 0.41473063  0.02890783  0.03133843 -0.10581102  0.29721554 -0.62980642
   0.45279739 -0.00892146  0.14673663 -0.32148853]<br>
 [-0.35940576 -0.44707915 -0.00590009  0.39774282  0.39934487  0.2109078
   0.32898666 -0.31867242 -0.15095844 -0.27601152]<br>
 [-0.01909706  0.51577804 -0.28417834  0.26775163  0.51450901 -0.0539428
  -0.50611417 -0.07537162 -0.00819261 -0.22722489]<br>
 [ 0.19767253 -0.25255201  0.18137541  0.55041273 -0.18490406  0.04076702
  -0.20581009  0.49476773  0.37238323 -0.31562352]<br>
 [-0.41985369  0.33739233  0.3507834  -0.26215878  0.14590668  0.15674749
   0.22509816  0.54012581 -0.14806143 -0.32852288]<br>
 [ 0.06927269 -0.503223   -0.16530012 -0.38951212  0.03453079 -0.14261544
  -0.43603674  0.16215967 -0.44620607 -0.35478007]<br>
 [ 0.00433431 -0.04353954 -0.53591166 -0.36968261 -0.01141732  0.4070828
   0.13224517 -0.00331918  0.5483555  -0.30029763]]<br>